In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable

In [ ]:
SEED = [1,2,3,4]
random = np.random.RandomState(SEED) 

In [ ]:
def f(x,y):
    return x % 1.0

In [ ]:
def visualise(f, minx=0.0, maxx=1.0, miny=0.0, maxy=1.0, width=256, height=256):
    xwidth = maxx - minx
    yheight = maxy - miny
    def get_pixel(row, column):
        x = column / (width-1) * xwidth + minx
        y = row / (height-1) * yheight + miny
        return f(x,y)
    pixels = np.flipud(np.fromfunction(np.frompyfunc(get_pixel,2,1), (height, width))).astype(float)*255
    return Image.fromarray(pixels).convert('L')

In [ ]:
visualise(f)

In [ ]:
visualise(lambda x,y:x+y)

In [ ]:
columns = 10
rows = 8

x = np.arange(0,columns,1)
y = np.arange(0,rows,1)

X, Y = np.meshgrid(x, y)

SEED = [1,2,3,4]
random = np.random.RandomState(SEED)

random_angles = random.rand(rows,columns) * (2 * np.pi)

x_components = np.cos(random_angles)
y_components = np.sin(random_angles)


In [ ]:
fig, ax = plt.subplots()

x_pos = X
y_pos = Y
x_direct = x_components
y_direct = y_components

assert x_pos.shape == y_pos.shape == x_direct.shape == y_direct.shape

ax.quiver(x_pos,y_pos,x_direct,y_direct, angles='xy', scale_units='xy', scale=2)

plt.show()

In [ ]:

def colour_point_by_gradient(x, y, gradient_column, gradient_row):
    gx = x_components[gradient_row, gradient_column]
    gy = y_components[gradient_row, gradient_column]
    dot_product = (x * gx) + (y * gy)

    return dot_product * 0.5 + 0.5

def f(x,y):
    return colour_point_by_gradient(x,y,3,5)

visualise(f, minx=-1.0, maxx=1.0, miny=-1.0, maxy=1.0)

In [ ]:
COLOUR_FUNC = Callable[[float,float],float]
GRID_FUNC = Callable[[int,int],COLOUR_FUNC]

def interpolate(f: GRID_FUNC) -> COLOUR_FUNC:
    def f_prime(x: float,y: float) -> float:
        grid_x = int(np.round(x))
        grid_y = int(np.round(y))

        pixel_func = f(grid_x,grid_y)
        return pixel_func(x,y)
    
    return f_prime

def gradient_grid_function(grid_x: int, grid_y: int) -> COLOUR_FUNC:
    def f(x,y):
        return colour_point_by_gradient(x - grid_x,y - grid_y ,grid_x,grid_y)

    return f

visualise(interpolate(gradient_grid_function), minx=0, maxx=5.0, miny=0, maxy=5.0)